Since the code was created by merging three different codes for processing data (biases, darks, flats) from three different nights, there are specific suffixes at the end of the variables to differenciate  which of the three nights they belong to:    
* `main` for the night on 20 April 2021
* `A39` for the night on 22 April 2021
* `old` for the night on 17 April 2020

Only frame A39 was taken during the night on 22 April 2021. The `old` images were taken by a different group last year.  
  
As said, the code was made by merging three different notebooks. Because of this, there are numerous functions defined that, even though they might not be strictly necessary, help to make the code more modular and universal in case changes had to be implemented.

## Import and prepare the data

In [ ]:
#import packages
from astropy.io import fits
import numpy as np
import os, fnmatch
from matplotlib.pyplot import show, subplots
from mpl_toolkits.axes_grid1 import make_axes_locatable
from IPython.display import clear_output
from image_registration import chi2_shift #non-default
from image_registration.fft_tools import shift #non-default
from astroscrappy import detect_cosmics #non-default

Import information neccessary to load the data.  
  
For each of the `main`, `A39` and `old` nights, the destination path for the images, the numbers of the images to exclude, and the numbers of the frames corresponding to the images taken that night were loaded.  
  
Additionaly, the `empty_slices.txt` file was loaded into an array. This file includes the indices of pixels needed to remove the backgroud light from the images.

In [ ]:
#main
destination_main = '/net/vega/data/users/observatory/images/210420/STL-6303E/i/'
exclude_main = np.array([31, 33, 34, 65])
frames_main = np.array(['A87', 'A47', 'A36', 'A38', 'A27', 'A16', 'A5', 'A26', 'A17', 'A28'])

#A39
destination_A39 = '/net/vega/data/users/observatory/images/210422/STL-6303E/i/'
exclude_A39 = np.arange(60,89)
frames_A39 = np.array(['A39'])

#old
destination_old = '/net/vega/data/users/observatory/images/200417/STL-6303E/i/'
exclude_old = np.append(np.arange(40,49), np.append(np.array([52]), np.append(np.arange(56, 65), np.arange(85,125))))
frames_old = np.array(['A50', 'A49', 'A48', 'A37'])

#info for background removal
empty_slices_path = '/net/virgo01/data/users/posker/ObservationalAstro/Project/empty_slices.txt'
empty_slices = np.genfromtxt(empty_slices_path, dtype='str', comments = "#", delimiter = ",")

Based on the user input, determine the correct night the frame was taken on and load the relevant data for that night.

In [ ]:
def is_in(frame, frame_names):
    '''Check if array "frame_names" includes value "frame". Return boolean value.'''
    i = np.where(frame_names == frame)[0]
    if i.size == 0:
        return False
    elif i.size > 0:
        return True

#ask user to imput the frame name that is about to be exported
frame_n = input('Enter frame to show and export (default A36): ')

#based on the user input, load the data from the relevant night the frame was taken on
#also determine the index of the frame within the frames array of the relevant night
if is_in(frame_n, frames_main):
    frame_index, destination, exclude = np.where(frames_main == frame_n)[0][0], destination_main, exclude_main
    night = '20/4/2021'
    print(f'Loading frame {frame_n} and the data from night on {night}...')
elif is_in(frame_n, frames_A39):
    frame_index, destination, exclude = np.where(frames_A39 == frame_n)[0][0], destination_A39, exclude_A39
    night = '22/4/2021'
    print(f'Loading frame {frame_n} and the data from night on {night}...')
elif is_in(frame_n, frames_old):
    frame_index, destination, exclude = np.where(frames_old == frame_n)[0][0], destination_old, exclude_old
    night = '17/4/2020'
    print(f'Loading frame {frame_n} and the data from night on {night}...')
#if the input is wrong in any way, take the default A36 frame
else:
    frame_n, frame_index, destination, exclude = 'A36', 2, destination_main, exclude_main
    night = '20/4/2021'
    print(f'Input "{frame_n}" not recognized, loading frame A36 from the night on 20/4/2021...')

#relevant header keywords and colors of the images in the correct order (for easier and shorter code later)
headers = ['IMAGETYP', 'EXPTIME', 'FILTER']
colors = ['B', 'V', 'R']

Based on the loaded destination paths and the numbers of the files to exclude, get all the names of the usable files, along with their full paths.

In [ ]:
def filenames(destination, exclude_numbers):
    '''Given destination folder and numbers of files to exclude, using function exclude_files, returns names of the non-excluded files and their full paths.'''
    
    def exclude_files(file_names, exclude_numbers):
        '''Excludes filenames with numbers specified in an array. Returns array with the filenames without the excluded files.'''
        
        #for each file, check if it is inluded in the exclude array, and if it is, remove it from the file_names
        for i in file_names:
            for exc in exclude_numbers:
                #this condition differentiates between numbers to be excluded that are lower and higher than 100, do add a zero to the string
                #does not support files with numbers larger than 999
                if exc < 100:    
                    if (f'.000000{exc}.' in i) == True:
                        file_names = file_names[file_names != i]
                else:
                    if (f'.00000{exc}.' in i) == True:
                        file_names = file_names[file_names != i]
        return file_names
    
    #import the sorted names of the fits files in the desired folder, without the excluded files
    file_names = exclude_files(np.sort(fnmatch.filter(os.listdir(destination), '*.FIT')), exclude_numbers)
    
    #generate another array, which includes the full file paths of all the files in the file_names array
    file_paths = np.array([f'{destination}{i}' for i in file_names])
    
    return file_names, file_paths

#execute the previously defined function to get the file names and their full paths
file_names, file_paths = filenames(destination, exclude)

Using the file paths and names, the extract function loops over the fits files and loads them. Then, it stacks the data into a 3D array and puts the necessary information, specified by the `headers` array, into a 2D array to be printed into a table and used later in the code.

In [ ]:
def extract(full_paths, file_names):
    '''Extract data of the files given as file names, stack data into 3D array and info about the frames into 2D array. Return both arrays.'''
    
    #empty arrays to be filled in the loop
    table_info = np.empty(0)
    data = np.empty(fits.open(full_paths[0])[0].data.shape)

    #the loop itself, filling the two arrays
    for path, name in zip(full_paths, file_names):
        #open fits
        hdulist = fits.open(path)
    
        #append info
        hdr = hdulist[0].header
        line = [name, hdr[headers[0]], hdr[headers[1]], hdr[headers[2]]]
        table_info = np.append(table_info, line)
    
        #stack data
        image_data = hdulist[0].data
        data = np.dstack((data, image_data))
        
        #continously print status of the loop
        clear_output(wait = True)
        if np.where(name == file_names)[0][0] < (file_names.shape[0] - 1):
            print(f'Loaded {np.where(name == file_names)[0][0]+1}/{file_names.shape[0]} frames...')
        else:
            print(f'Done! Loaded all {file_names.size}/{file_names.size} frames')
    
    #format the arrays from the loop
    data = data[:,:,1:] #remove empty first image
    table_info = table_info.reshape(file_names.size, len(line)) #reshape
    
    return data, table_info

#use the extract function to get the data in a 3D array and the table_info in a 2D array
data, table_info = extract(file_paths, file_names)

Slice the `table_info` into separate 1D arrays. Print a table listing all the relevant files and the inforation about them.

In [ ]:
def table_values(table_info):
    '''Separates the array with the info for the table into separate arrays.'''
    
    #slice the table_info array; exposures are loaded as float
    image_types, exposures, filters = table_info[:,1], table_info[:,2].astype('float'), table_info[:,3]
    
    return image_types, exposures, filters

def print_table(file_names, image_types, exposures, filters):
    '''Prints a table with the information from the given arrays.'''
    print('| {0:^52} | {1:^15} | {2:^10} | {3:^8} |'.format('File name', 'Image type', 'Exp. time', 'Filter'))
    print('----------------------------------------------------------------------------------------------')
    
    #print the info about the files using a loop
    for i in range(len(file_names)):
        print('| {0:^52} | {1:^15} | {2:>10} | {3:>8} |'.format(file_names[i], image_types[i], exposures[i], filters[i]))

#use the table_values function to extract the individual properties of the files into separate 1D arrays
image_types, exposures, filters = table_values(table_info)

#print the table using the print_table function, given the previously obtained 1D arrays with the information
print_table(file_names, image_types, exposures, filters)

Separate the `data` array based on the purpose of the images.

In [ ]:
def separate(data, image_types, filters):
    '''Separates the whole data array into 3D arrays of frames based on their function. 
    Returns the individual arrays (flats arrays are grouped into a touple based on the color).'''
    bias_frames = data[:,:,image_types == 'Bias Frame']
    dark_frames = data[:,:,image_types == 'Dark Frame']

    flat_frames_B = data[:,:,(image_types == 'Flat Field') & (filters == 'B')]
    flat_frames_V = data[:,:,(image_types == 'Flat Field') & (filters == 'V')]
    flat_frames_R = data[:,:,(image_types == 'Flat Field') & (filters == 'R')]

    images = data[:,:,image_types == 'Light Frame']
    
    return bias_frames, dark_frames, (flat_frames_B, flat_frames_V, flat_frames_R), images

#use the slice function to separate the images in the data array based on their function
bias_frames, dark_frames, flat_frames, images = separate(data, image_types, filters)

#separate the tuple with flats based on color
flat_frames_B, flat_frames_V, flat_frames_R = flat_frames

## Bias

Calculate master bias `B` for the relevant night.

In [ ]:
#calculate master bias by taking the median of the bias frames for every separate pixel
B = np.median(bias_frames, axis = -1)

Define three functions used from now on to plot images and work with data.

In [ ]:
def zero_floor(array):
    '''Makes all negative values in an array equal to zero. Input can also be just a number.'''
    array = np.where(array < 0, 0, array)
    return array

def vmax(image):
    '''Defines vmax for plotting an image as median of the pixels + standard deviation.'''
    vmax = np.median(image) + image.std()
    return vmax

def vmin(image):
    '''Defines vmin for plotting an image as median of the pixels - standard deviation. If the result is lower than zero, returns zero.'''
    vmin = zero_floor(np.median(image) - image.std())
    return vmin

Plot the calculated master bias for the relevant night.

In [ ]:
fig, frame = subplots(figsize = (12,12))
image = frame.imshow(B, vmax = vmax(B), vmin = vmin(B), cmap = 'binary_r', origin = "lower")
divider = make_axes_locatable(frame)
colbarframe = divider.append_axes("right", size = "5%", pad = 0.15)
cbar = fig.colorbar(image, ax = frame, cax = colbarframe)
frame.set_xlabel('x [px]'), frame.set_ylabel('y [px]')
frame.set_title(f'Master Bias from {night}')
fig.tight_layout()
show()

## Dark

Calculate the master dark normalized per second exposure `Dn` for the relevant night.

In [ ]:
def corr_dark(dark_frames, B, exposures, image_types):
    '''Corrects given darks for bias, calculates the master dark and normalizes it per second of exposure.'''
    
    def bias_correction(dark_frames, B):
        '''Corrects individual dark frames from the 3D array for the master bias.'''
        
        #loop through all the individual dark frames and correct each one for master bias
        for i in range(dark_frames.shape[2]):
            dark_frames[:,:,i] = dark_frames[:,:,i] - B
        
        return dark_frames
    
    def D_norm(D, exposures, image_types):
        '''Calculates the master dark normalized per second of exposure.'''
        
        #get the exposure time of the darks
        #this assumes that all the darks for the relevant night have the same exposure time
        exp_dark = int(exposures[image_types == 'Dark Frame'][0])
        
        #normalize the master dark per second exposure
        Dn = D/exp_dark
        
        return Dn
        
    #use the bias_correction fucntion to correct every individual dark frame for master bias
    dark_frames = bias_correction(dark_frames, B)
    
    #calculate the master dark by taking the median of the dark frames for every separate pixel
    #use the zero_floor function to make sure there are no negative pixels
    D = zero_floor(np.median(dark_frames, axis = -1))
    
    #use the D_norm function to normalize the master dark per second exposure
    Dn = D_norm(D, exposures, image_types)

    return Dn

#use the corr_dark function to calculates the master dark normalized per second exposure
Dn = corr_dark(dark_frames, B, exposures, image_types)

Plot the calculated master dark normalized per second exposure for the relevant night.

In [ ]:
fig, frame = subplots(figsize = (12,12))
image = frame.imshow(Dn, vmax = vmax(Dn), vmin = vmin(Dn), cmap = 'binary_r', origin = "lower")
divider = make_axes_locatable(frame)
colbarframe = divider.append_axes("right", size = "5%", pad = 0.15)
cbar = fig.colorbar(image, ax = frame, cax = colbarframe)
frame.set_xlabel('x [px]'), frame.set_ylabel('y [px]')
frame.set_title(f'Master Dark from {night}')
fig.tight_layout()
show()

## Flats

Calculate the master flat `Fn` for each separate color and stack the three flats into a 3D array.

In [ ]:
def flats(image_types, filters, exposures, B, Dn, flats_B, flats_V, flats_R):
    '''Given arrays with info about the frames, the master bias, the master dark adjusted for exposure
    and 3D arrays with separate color flat frames; returns mater flats for each of the three colors.'''
    
    def corr_flat(flats, index_loop, indices, B, Dn, exposures):
        '''Function to correct the individual flats within each array for bias and dark. Made to be used
        in the loop in within function flats() below. Returns corrected flat.'''
        
        #correct the individual flat for master bias and master dark adjusted for exposure of the flat
        #index_loop-indices.min() is a way to make the index start from zero to correctly get the frame from the array of flats of that color
        corrected_flat = flats[:,:,index_loop-indices.min()] - B - (Dn * exposures[index_loop])
        
        #divide the flat by its median in order for the median of the flat to be 1
        flats[:,:,index_loop-indices.min()] = corrected_flat/np.median(corrected_flat)
        
        return flats
        
    def indices_flats(color):
        '''Gives an array with indices of flat frames with given color. Input "color" has to be string. '''
        indices = np.where((image_types == 'Flat Field') & (filters == color))[0]
        return indices

    #indices of the flats from the original data table
    indices_flats_B, indices_flats_V, indices_flats_R = indices_flats('B'), indices_flats('V'), indices_flats('R')
    
    #loop over the individual flats of each color and make the necessary adjustments (subtract B and Dn adjusted for exposure)
    #designed in a way that only works if there is the same number of flat frames for every color
    for b, v, r in zip(indices_flats_B, indices_flats_V, indices_flats_R):
        flats_B = corr_flat(flats_B, b, indices_flats_B, B, Dn, exposures)
        flats_V = corr_flat(flats_V, v, indices_flats_V, B, Dn, exposures)
        flats_R = corr_flat(flats_R, r, indices_flats_R, B, Dn, exposures)
    
    #calculate master flat of each filter (median of the relevant flats for each separate pixel)
    Fn_B, Fn_V, Fn_R = np.median(flats_B, axis = -1), np.median(flats_V, axis = -1), np.median(flats_R, axis = -1)
    
    #create a 3D array with the three flats stacked on top of each other
    stacked_flats = np.dstack((Fn_B, np.dstack((Fn_V, Fn_R))))
    
    return stacked_flats

#use the flats
flats_BVR = flats(image_types, filters, exposures, B, Dn, flat_frames_B, flat_frames_V, flat_frames_R)

Plot the calculated master flats for the relevant night. Each plot is of a flat for a separate color of the filter.

In [ ]:
fig, frame = subplots(figsize = (12,24))
frame.axis('off')
for i in range(flats_BVR.shape[2]):
    frame = fig.add_subplot(3,1,i+1)
    image = frame.imshow(flats_BVR[:,:,i], vmax = vmax(flats_BVR[:,:,i]), vmin = vmin(flats_BVR[:,:,i]), cmap = 'binary_r', origin = "lower")
    divider = make_axes_locatable(frame)
    colbarframe = divider.append_axes("right", size = "5%", pad = 0.15)
    cbar = fig.colorbar(image, ax = frame, cax = colbarframe)
    frame.set_xlabel('x [px]'), frame.set_ylabel('y [px]')
    frame.set_title(f'Flat {colors[i]} from {night}')
fig.tight_layout()
show()

## Images

Get the BVR images from the `images` array, correct each of them separately for master bias, master dark and master flat. Also normalize the exposure so that all three images would be identically exposed. Stack the three images into a 3D array. Remove majority of cosmic rays in the images by looping over the stack.

In [ ]:
def process(images, exposures, image_types, frame_index, B, Dn, flats_BVR): # +1D array coordinates + frame_n
    '''Given all the images in an array, the exposures, image types, the index of the required frame and the correction 
    frames (master bias, master dark, flats), returns a 3D array of the stacked BVR images of the chosen frame.'''
    
    def correction(image, exp, B, Dn, Fn, frame_n, empty_slices):
        '''Corrects individual image for different exposure, master bias, master dark, master flat and the background light.'''
        
        def background_corr(image, frame_n, empty_slices):
            '''Correct the image for the background light based on the indices given in empty_slices.'''
            
            #load the line of coordinates (indices) relevant for the frame, convert them to integers
            coordinates = empty_slices[frame_n == empty_slices[:,0],1:][0].astype(int)
            
            #create empty array to be filled in the loop
            background = np.empty(0)
            
            #loop over the four slices without stars in them and append all their pixels into an array
            #since only median of all the mixels is necessary, it is ok for the array to be flattened
            for i in range(int(coordinates.size/4)):
                #append the pixels from the slice of the image
                background = np.append(background, image[coordinates[4*i]:coordinates[(4*i)+1], coordinates[(4*i)+2]:coordinates[(4*i)+3]])
            
            #calculate the median of all four of the slices
            background_med = np.median(background)
            
            #correct the image for the background light by subtracting the median of the pixels from the areas without any stars
            image = zero_floor(image - background_med)
            
            return image
        
        #correct the image for different exposure, master bias, master dark, master flat and background light
        image = background_corr(zero_floor(((image * (300 / exp)) - B - (Dn * 300))/Fn), frame_n, empty_slices)
        
        return image
    
    #calculate the indices of each of the BVR images from the frame_index, get the exposures for the light frames
    frame_indices, exp_light_frames = [3*frame_index, (3*frame_index) + 1, (3*frame_index) + 2], exposures[image_types == 'Light Frame']
    
    #use the correction function to do all the necessary corrections to each of the BVR images
    image_B = correction(images[:,:,frame_indices[0]], exp_light_frames[frame_indices[0]], B, Dn, flats_BVR[:,:,0], frame_n, empty_slices)
    image_V = correction(images[:,:,frame_indices[1]], exp_light_frames[frame_indices[1]], B, Dn, flats_BVR[:,:,1], frame_n, empty_slices)
    image_R = correction(images[:,:,frame_indices[2]], exp_light_frames[frame_indices[2]], B, Dn, flats_BVR[:,:,2], frame_n, empty_slices)
    
    #stack the images into 3D array and stack them vertically
    image_BVR = np.flip(np.dstack((image_B, np.dstack((image_V, image_R)))), axis = 0) #flipped!
    
    #loop over the images and correct each of them for cosmic rays
    for i in range(3):
        mask, clean = detect_cosmics(image_BVR[:,:,i], gain = 1.4, readnoise = 15.2, sigclip = 2)
        image_BVR[:,:,i] = clean
   
    return image_BVR

#use the process funtion to get the 3D array of the processed BVR images of the chosen frame
image_BVR = process(images, exposures, image_types, frame_index, B, Dn, flats_BVR)

Plot the obtained BVR images. Each plot is of an image for a separate color of the filter.

In [ ]:
fig, frame = subplots(figsize = (12,24))
frame.axis('off')
for i in range(image_BVR.shape[2]):
    frame = fig.add_subplot(3,1,i+1)
    image = frame.imshow(image_BVR[:,:,i], vmax = vmax(image_BVR[:,:,i]), vmin = vmin(image_BVR[:,:,i]), cmap = 'binary_r', origin = "lower")
    divider = make_axes_locatable(frame)
    colbarframe = divider.append_axes("right", size = "5%", pad = 0.15)
    cbar = fig.colorbar(image, ax = frame, cax = colbarframe)
    frame.set_xlabel('x [px]'), frame.set_ylabel('y [px]')
    frame.set_title(f'{frame_n} in {colors[i]} from {night}')
fig.tight_layout()
show()

Ask the user whether or not to export the result (BVR images) into separate fits files. If given "y", export the files.

In [ ]:
def export(image, file_paths, frame_n, image_types, filters, color):
    '''Exports the given image into a fits file.'''
    
    #import the image as the data file of a new fits file
    new_hdulist = fits.PrimaryHDU(image)
    
    #copy header information from a target frame of the same color
    new_hdulist.header = fits.open(file_paths[(image_types == 'Light Frame') & (filters == color)][0])[0].header
    
    #change image type to target
    new_hdulist.header['IMAGETYP'] = 'Target'
    
    #export the fits file
    new_hdulist.writeto(f'{frame_n}_{color}.fits', overwrite = True, output_verify='ignore')

#get user input whether or not to export the image
y_n = input(f'Export the BVR images of {frame_n} as fits? (y/n) ')

if y_n == 'y':
    
    #loop over the three images to export each of them separately
    for i in range(3):
        export(image_BVR[:,:,i], file_paths, frame_n, image_types, filters, colors[i])
    
    print(f'Exported BVR frames of {frame_n} as {frame_n}_B.fits, {frame_n}_V.fits and {frame_n}_R.fits')
else:
    print('Frames were not exported')